In [81]:
import pandas as pd
import numpy as np
import cvxpy as cp
from scipy.linalg import block_diag

# -----------------------------------------------------------
# 1. Load data  (each cell is already a wealth multiplier 1+r)
# -----------------------------------------------------------
data = pd.read_csv("3DailyReturns.csv",
                   index_col=0,               # dates in column 0
                   usecols=[1,2,3,4,5,6,7,8]) # 8 assets
data

OSError: [Errno 22] Invalid argument: '3DailyReturns.csv'

In [ ]:
# Split the sample in half
length  = data.shape[0] // 2
data1   = data.iloc[:length]      # first sub-sample
data2   = data.iloc[length:]      # second sub-sample

# One-day simple returns and their covariances
R1  = data1 - 1.0                 # simple returns in half 1
R2  = data2 - 1.0                 # simple returns in half 2

cov1 = R1.cov() * length          # scale by horizon (≈ “variance over half-sample”)
cov2 = R2.cov() * length

# 4. Period-aggregated gross-return factors
a1 = data1.prod()                 # Π(1+r) over first half
a2 = data2.prod()                 # Π(1+r) over second half

phi_1_2 = a1 * a2                 # factors over the *full* horizon
phi_2_2 = a2.copy()               # second period only (copied for clarity)

# (Optional) stack them for later use
a = np.concatenate([phi_1_2.values, phi_2_2.values])

# -----------------------------------------------------------
# 5. Block-diagonal risk matrix Q  (no cross-period covariance)
# -----------------------------------------------------------
Q = block_diag(cov1.values, cov2.values)

# -----------------------------------------------------------
# 6. Two-stage mean–variance optimisation
# -----------------------------------------------------------
n_assets = data.shape[1]
T        = 2

# decision vector w = [w₁, w₂]  (length 2·n_assets)
w = cp.Variable(n_assets * T)

objective   = cp.Minimize(cp.quad_form(w, Q))
constraints = []

for k in range(T):
    s, e = k * n_assets, (k + 1) * n_assets
    if k == 0:                         # first-period portfolio fully invested
        constraints.append(cp.sum(w[s:e]) == 1)
    else:                              # second-period trade self-financing
        constraints.append(cp.sum(w[s:e]) == 0)

prob = cp.Problem(objective, constraints)
prob.solve()

# -----------------------------------------------------------
# 7. Results
# -----------------------------------------------------------
w_opt = w.value
w1, w2 = w_opt[:n_assets], w_opt[n_assets:]

print("Optimal weights at start of period 1:\n", w1)
print("\nOptimal *self-financing* trade for period 2:\n", w2)

# Optional: scenario gross return of the two-period strategy
scenario_gross = np.dot(phi_1_2.values, w1) + np.dot(phi_2_2.values, w2)
print("Return:", round(scenario_gross-1, 3))
print("Minimum Std", round(np.sqrt(prob.value),3))

Optimal weights at start of period 1:
 [ 0.103441  0.328256  0.017528  0.022283 -0.046695 -0.035022  0.61021 ]

Optimal *self-financing* trade for period 2:
 [0. 0. 0. 0. 0. 0. 0.]
Return: 0.095
Minimum Std 0.116


In [ ]:
# ===========================================================
# Helper
# ===========================================================
def solve_exact(Q, mu_vec, n_assets, target):
    T = Q.shape[0] // n_assets
    w = cp.Variable(Q.shape[0])

    cons = [cp.sum(w[0:n_assets]) == 1]          # fully invested
    for k in range(1, T):                        # self-financing trades
        cons.append(cp.sum(w[k*n_assets:(k+1)*n_assets]) == 0)
    cons.append(mu_vec @ w == target)            # equality target

    prob = cp.Problem(cp.Minimize(cp.quad_form(w, Q)), cons)
    prob.solve()
    return w.value, np.sqrt(prob.value)

# ===========================================================
# 0.  Load wealth-multiplier data (each cell is 1+r_d)
# ===========================================================
data = pd.read_csv("3DailyReturns.csv", index_col=0, usecols=range(1,9))
n_assets      = data.shape[1]
target_return = 0.3145          # 31.45 % simple return over the horizon

# --------------------------- 2 HALF-YEAR MODEL ----------------------------
Lh      = data.shape[0] // 2
H       = [data.iloc[i*Lh:(i+1)*Lh] for i in range(2)]
R_half  = [h - 1.0 for h in H]

Σ_half  = [R.cov()*Lh          for R in R_half]
μ_half  = [R.mean().values*Lh  for R in R_half]

μ_vec_2 = np.concatenate([μ_half[0] + μ_half[1],      # w1 exposed in both halves
                          μ_half[1]])                 # u2 exposed only in half-2
Σ11_2   = Σ_half[0] + Σ_half[1]                       # var(r_H1+r_H2) no serial corr
Q_2     = block_diag(Σ11_2.values, Σ_half[1].values)

w2, σ2  = solve_exact(Q_2, μ_vec_2, n_assets, target_return)
w1, u2  = w2[:n_assets], w2[n_assets:]

# one-path (in-sample) gross factors
a1, a2        = [h.prod() for h in H]
phi_1_2       = a1 * a2
phi_2_2       =       a2
scenario_ret2 = phi_1_2 @ w1 + phi_2_2 @ u2 - 1

# --------------------------- 4 QUARTER MODEL ------------------------------
Tq      = 4
Lq      = data.shape[0] // Tq
Qtr     = [data.iloc[i*Lq:(i+1)*Lq] for i in range(Tq)]
R_q     = [Q - 1.0 for Q in Qtr]

Σ_q     = [R.cov()*Lq          for R in R_q]
μ_q     = [R.mean().values*Lq  for R in R_q]

μ_blocks_4 = [μ_q[0]+μ_q[1]+μ_q[2]+μ_q[3],
              μ_q[1]+μ_q[2]+μ_q[3],
              μ_q[2]+μ_q[3],
              μ_q[3]]
Σ_blocks_4 = [Σ_q[0]+Σ_q[1]+Σ_q[2]+Σ_q[3],
              Σ_q[1]+Σ_q[2]+Σ_q[3],
              Σ_q[2]+Σ_q[3],
              Σ_q[3]]

μ_vec_4 = np.concatenate(μ_blocks_4)
Q_4     = block_diag(*(S.values for S in Σ_blocks_4))

w4, σ4  = solve_exact(Q_4, μ_vec_4, n_assets, target_return)
w0 = w4[0:n_assets]
u1 = w4[  n_assets:2*n_assets]
u2 = w4[2*n_assets:3*n_assets]
u3 = w4[3*n_assets:4*n_assets]

# one-path gross factors for quarters
a_q = [Q.prod() for Q in Qtr]
phi_q = [a_q[0]*a_q[1]*a_q[2]*a_q[3],
              a_q[1]*a_q[2]*a_q[3],
                     a_q[2]*a_q[3],
                            a_q[3]]
scenario_ret4 = (phi_q[0] @ w0 + phi_q[1] @ u1 +
                 phi_q[2] @ u2 + phi_q[3] @ u3 - 1)

# ----------------------------  PRINT  ------------------------------------
np.set_printoptions(precision=6, suppress=True)

print("\n=============== TWO-STAGE (HALF-YEAR) ===============")
print("Expected return (μ·w):", round(target_return,4))
print("Scenario simple return:", round(scenario_ret2,4))
print("Minimum std           :", round(σ2,4))
print("w1 :", w1)
print("u2 :", u2)

print("\n=============== FOUR-STAGE (QUARTERLY) ==============")
print("Expected return (μ·w):", round(target_return,4))
print("Scenario simple return:", round(scenario_ret4,4))
print("Minimum std           :", round(σ4,4))
print("w0 :", w0)
print("u1 :", u1)
print("u2 :", u2)
print("u3 :", u3)

print("\n=============  SUMMARY  =============")
print(f"σ (2-stage) : {σ2:5.4f}")
print(f"σ (4-stage) : {σ4:5.4f}")


=============== TWO-STAGE (HALF-YEAR) ===============
Expected return (μ·w): 0.3145
Scenario simple return: 0.3252
Minimum std           : 0.1827
w1 : [ 0.062908  0.380278  0.093991 -0.003049  0.040561  0.005607  0.419704]
u2 : [ 0.031529  0.085549 -0.086959  0.031955  0.067593  0.001753 -0.131421]

=============== FOUR-STAGE (QUARTERLY) ==============
Expected return (μ·w): 0.3145
Scenario simple return: 0.3116
Minimum std           : 0.1805
w0 : [ 0.056062  0.372297  0.090663 -0.006078  0.019678 -0.016063  0.483441]
u1 : [ 0.007873  0.036048  0.018535  0.011671  0.015558  0.025349 -0.115033]
u2 : [ 0.031529  0.085549 -0.086959  0.031955  0.067593  0.001753 -0.131421]
u3 : [ 0.092131 -0.002262 -0.073233  0.033061  0.08253   0.002311 -0.134539]

=============  SUMMARY  =============
σ (2-stage) : 0.1827
σ (4-stage) : 0.1805


In [ ]:
def solve_exact(Q, mu_vec, n_assets, target):
    T = Q.shape[0] // n_assets
    w = cp.Variable(Q.shape[0])
    cons = [cp.sum(w[0:n_assets]) == 1]
    for k in range(1, T):
        cons.append(cp.sum(w[k*n_assets:(k+1)*n_assets]) == 0)
    cons.append(mu_vec @ w == target)
    prob = cp.Problem(cp.Minimize(cp.quad_form(w, Q)), cons)
    prob.solve()
    return w.value, np.sqrt(prob.value)

# ----------------- Load data -------------------
data = pd.read_csv("3DailyReturns.csv", index_col=0, usecols=range(1, 9))
n_assets = data.shape[1]
target_return = 0.5526

# ----------------- 2-STAGE (Half-Year) -------------------
Lh = data.shape[0] // 2
H = [data.iloc[i*Lh:(i+1)*Lh] for i in range(2)]

# Use full-year estimates, scaled
excess_returns = data - 1.0
mu_year = excess_returns.mean().values * data.shape[0]
cov_year = excess_returns.cov() * data.shape[0]

mu_vec_2 = np.concatenate([
    mu_year * (2/2),  # stage 1
    mu_year * (1/2)   # stage 2
])
Q_2 = block_diag(
    cov_year.values * (2/2),
    cov_year.values * (1/2)
)

w2, σ2 = solve_exact(Q_2, mu_vec_2, n_assets, target_return)
w1, u2 = w2[:n_assets], w2[n_assets:]
a1, a2 = [h.prod().values for h in H]
phi_1_2 = a1 * a2
phi_2_2 = a2
scenario_ret2 = phi_1_2 @ w1 + phi_2_2 @ u2 - 1

# ----------------- 4-STAGE (Quarterly) -------------------
Tq = 4
Lq = data.shape[0] // Tq
Qtr = [data.iloc[i*Lq:(i+1)*Lq] for i in range(Tq)]

mu_vec_4 = np.concatenate([
    mu_year * (4/4),
    mu_year * (3/4),
    mu_year * (2/4),
    mu_year * (1/4)
])
Q_4 = block_diag(
    cov_year.values * (4/4),
    cov_year.values * (3/4),
    cov_year.values * (2/4),
    cov_year.values * (1/4)
)

w4, σ4 = solve_exact(Q_4, mu_vec_4, n_assets, target_return)
w0 = w4[0:n_assets]
u1 = w4[n_assets:2*n_assets]
u2 = w4[2*n_assets:3*n_assets]
u3 = w4[3*n_assets:4*n_assets]
a_q = [Q.prod().values for Q in Qtr]
phi_q = [
    a_q[0]*a_q[1]*a_q[2]*a_q[3],
    a_q[1]*a_q[2]*a_q[3],
    a_q[2]*a_q[3],
    a_q[3]
]
scenario_ret4 = phi_q[0] @ w0 + phi_q[1] @ u1 + phi_q[2] @ u2 + phi_q[3] @ u3 - 1

# ----------------- EQUAL-WEIGHTED -------------------
w_eq = np.ones(n_assets) / n_assets
R_full = data - 1.0
gross_eq = data.prod().values
final_wealth_eq = w_eq @ gross_eq
scenario_ret_eq = final_wealth_eq - 1
σ_eq = np.sqrt(w_eq @ cov_year.values @ w_eq)

# ----------------- Sharpe Ratios -------------------
rf = 0.0472
sharpe_2 = (scenario_ret2 - rf) / σ2
sharpe_4 = (scenario_ret4 - rf) / σ4
sharpe_eq = (scenario_ret_eq - rf) / σ_eq

# ----------------- SINGLE PERIOD OPTIMISATION (SPO) -------------------
data_single = data - 1
simpleReturn = pd.read_csv("2SimpleReturn.csv", usecols=[1])
simpleReturn = simpleReturn.iloc[:, 0].to_numpy() / 100

mean_sp = data_single.mean() * 252
cov_sp = data_single.cov() * 252

w_sp = cp.Variable(n_assets)
objective_sp = cp.Minimize(cp.quad_form(w_sp, cov_sp.values))
constraints_sp = [
    cp.sum(w_sp) == 1,
    simpleReturn @ w_sp == target_return
]
prob_sp = cp.Problem(objective_sp, constraints_sp)
prob_sp.solve()

ret_sp = float(simpleReturn @ w_sp.value)
σ_sp = np.sqrt(w_sp.value @ cov_sp.values @ w_sp.value)
sharpe_sp = (ret_sp - rf) / σ_sp

# ----------------- PRINT -------------------
print("\n=============== SINGLE PERIOD OPTIMISATION ============")
print("Expected return :", round(target_return, 4))
print("Scenario return :", round(ret_sp, 4))
print("Std Dev         :", round(σ_sp, 4))
print("Weights         :", w_sp.value)

# ----------------- PRINT -------------------
np.set_printoptions(precision=6, suppress=True)

print("\n=============== TWO-PERIOD (HALF-YEAR) ===============")
print("Expected return :", round(target_return, 4))
print("Scenario return :", round(scenario_ret2, 4))
print("Std Dev         :", round(σ2, 4))
print("w1 :", w1)
print("u2 :", u2)

print("\n=============== FOUR-PERIOD (QUARTERLY) ==============")
print("Expected return :", round(target_return, 4))
print("Scenario return :", round(scenario_ret4, 4))
print("Std Dev         :", round(σ4, 4))
print("w0 :", w0)
print("u1 :", u1)
print("u2 :", u2)
print("u3 :", u3)

print("\n=============== EQUAL WEIGHTED ======================")
print("Scenario return :", round(scenario_ret_eq, 4))
print("Std Dev         :", round(σ_eq, 4))

# Append to summary
print("\n=============  SUMMARY  =============")
print(f"Std Dev (2-period) : {σ2:5.4f}")
print(f"Std Dev (4-period) : {σ4:5.4f}")
print(f"Std Dev (SPO)      : {σ_sp:5.4f}")
print(f"Equal-weighted     : {σ_eq:5.4f}\n")

print(f"Sharpe (2-period) : {sharpe_2:.4f}")
print(f"Sharpe (4-period) : {sharpe_4:.4f}")
print(f"Sharpe (SPO)      : {sharpe_sp:.4f}")
print(f"Sharpe (Equal-w)  : {sharpe_eq:.4f}")


=============== SINGLE PERIOD OPTIMISATION ============
Expected return : 0.5526
Scenario return : 0.5526
Std Dev         : 0.2131
Weights         : [ 0.059211  0.432358  0.095959 -0.009698  0.126999  0.172983  0.122188]

=============== TWO-PERIOD (HALF-YEAR) ===============
Expected return : 0.5526
Scenario return : 0.6127
Std Dev         : 0.2202
w1 : [0.093777 0.432219 0.112414 0.013944 0.149084 0.119508 0.079053]
u2 : [ 0.027687  0.045983  0.017477  0.015005  0.09839   0.104169 -0.308711]

=============== FOUR-PERIOD (QUARTERLY) ==============
Expected return : 0.5526
Scenario return : 0.5349
Std Dev         : 0.2032
w0 : [0.075319 0.401564 0.100762 0.003941 0.083491 0.050063 0.28486 ]
u1 : [ 0.027687  0.045983  0.017477  0.015005  0.09839   0.104169 -0.308711]
u2 : [ 0.027687  0.045983  0.017477  0.015005  0.09839   0.104169 -0.308711]
u3 : [ 0.027687  0.045983  0.017477  0.015005  0.09839   0.104169 -0.308711]

=============== EQUAL WEIGHTED ======================
Scenario retu